In [11]:
# Import py_entitymatching package
import py_entitymatching as em
import os
import pandas as pd
import numpy as np

In [20]:
# Load the pre-labeled data
metacriticData = em.read_csv_metadata("data/metacritic.csv")
wikiData = em.read_csv_metadata("data/wikiData.csv")

# add ID column to each dataset
metacriticID = ["a" + str(num) for num in np.arange(1, len(metacriticData.index)+1)]
wikiID = ["b" + str(num) for num in np.arange(1, len(wikiData.index)+1)]

col_idx = 0
metacriticData.insert(loc = col_idx, column = 'ID', value = metacriticID)
wikiData.insert(loc = col_idx, column = 'ID', value = wikiID)
em.set_key(wikiData, 'ID')
em.set_key(metacriticData, 'ID')

em.set_key(wikiData, 'ID')
em.set_key(metacriticData, 'ID')

True

In [50]:
#read in labeled samples
S = em.read_csv_metadata("candidates_sample.csv", 
                         key='_id',
                         ltable=metacriticData, rtable=wikiData, 
                         fk_ltable='ltable_ID', fk_rtable='rtable_ID')
S.head()

,Unnamed: 0,_id,ltable_ID,rtable_ID,ltable_Album,ltable_Artist,ltable_Release Date,rtable_Album,rtable_Artist,rtable_Release Date,label
0,24,24,a1024,b2294,world eater,blanck mass,Mar 3 2017,world eater,blanck mass,Mar 3 2017,1
1,83,83,a1074,b2263,youngish american,dams of the west,Feb 24 2017,still // alone,the golden filter,Feb 24 2017,0
2,98,98,a1094,b2236,terrible human beings,the orwells,Feb 17 2017,terrible human beings,the orwells,Feb 17 2017,1
3,127,127,a1116,b320,new order presents be music,various artists,Feb 17 2017,i'll be home for christmas,various artists,Nov 24 2014,0
4,150,150,a1143,b2171,process,sampha,Feb 3 2017,process,sampha,Feb 3 2017,1


In [52]:
# Split S into I an J
i_file = "I.csv"
j_file = "J.csv"
if not os.path.isfile(i_file): #so you don't delete your labels on accident
    IJ = em.split_train_test(S, train_proportion=0.5, random_state=0)
    I = IJ['train']
    J = IJ['test']
    I.to_csv(i_file,sep=",")
    J.to_csv(j_file,sep=",")
    print "Split samples into I and J"
else:
    I = em.read_csv_metadata(i_file)
    J = em.read_csv_metadata(j_file)
    print "Reading I and J from files"

Reading I and J from files
